In [47]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import explained_variance_score, r2_score
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from scipy.stats import pearsonr
import scipy.stats as stats

import seaborn as sns
import matplotlib.pyplot as plt

# 1 Preparation

## 1.1 Embeddings

In [2]:
embeddings = pd.read_pickle('test/embeddings_complete.pickle')
len(embeddings)

10060

In [3]:
# combine 2 embeddings of father and mother for each individual
sample_id = []
gene_id = []
embedding = []
for i in range(0, len(embeddings), 2):
    sid = embeddings[i][0].split(':')[1]
    gid = embeddings[i][0].split(':')[2]
    sample_id.append(sid)
    gene_id.append(gid)
    combined_embedding = np.concatenate([embeddings[i][1], embeddings[i+1][1]]) # concatenate 2 embeddings.
    embedding.append(combined_embedding)
df = pd.DataFrame({"sample_id": sample_id, "gene_id": gene_id, "embedding": embedding})

In [4]:
df.head()

,sample_id,gene_id,embedding
0,NA20795,ENSG00000198502.5,"[0.017137265, -0.009090343, 0.0061443364, -0.0..."
1,HG00260,ENSG00000214425.1,"[0.05563126, 0.00049458974, -0.0046671517, -0...."
2,HG01632,ENSG00000176681.9,"[-0.051695395, 0.023533892, -0.04169209, 0.067..."
3,HG00173,ENSG00000238083.3,"[0.042141862, 0.0355676, 0.023393063, -0.03903..."
4,HG00178,ENSG00000229450.2,"[0.0736973, -0.022161566, -0.008337349, -0.040..."


## 1.2 Expression Data

In [5]:
expression = pd.read_csv("datasets/GD660.GeneQuantRPKM.txt.gz", sep="\t")
expression.head()

,TargetID,Gene_Symbol,Chr,Coord,HG00096.1.M_111124_6,HG00097.7.M_120219_2,HG00099.1.M_120209_6,HG00099.5.M_120131_3,HG00100.2.M_111215_8,HG00101.1.M_111124_4,...,NA20810.2.M_111215_7,NA20811.1.M_111124_5,NA20812.2.M_111216_6,NA20813.5.M_120131_1,NA20814.2.M_111215_6,NA20815.5.M_120131_5,NA20816.3.M_120202_7,NA20819.3.M_120202_2,NA20826.1.M_111124_1,NA20828.2.M_111216_8
0,ENSG00000225538.1,ENSG00000225538.1,11,55850277,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.0000,0.00000,0.00000,0.00000,0.0000,0.00000,0.00000,0.03876,0.00000
1,ENSG00000237851.1,ENSG00000237851.1,6,143109260,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.0000,0.00000,0.00000,0.00000,0.0000,0.00000,0.00000,0.00000,0.00000
2,ENSG00000243765.1,ENSG00000243765.1,15,58442766,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.0000,0.00000,0.00000,0.00000,0.0000,0.00000,0.00000,0.00000,0.00000
3,ENSG00000257527.1,ENSG00000257527.1,16,18505708,0.70561,0.66697,0.64004,0.26195,0.34695,1.49208,...,0.87085,0.9495,0.95837,0.51002,0.29422,0.2296,0.58671,0.27674,0.53630,0.17139
4,ENSG00000212855.5,ENSG00000212855.5,Y,9578193,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.0000,0.00000,0.00000,0.00000,0.0000,0.00000,0.00000,0.00000,0.00000


In [6]:
expression = expression[expression["Gene_Symbol"].isin(df["gene_id"].values)]
expression = pd.melt(expression, id_vars=["TargetID", "Gene_Symbol", "Chr", "Coord"], var_name="sample_id", value_name="expression")
expression["sample_id"] = expression["sample_id"].str.split(".", expand=True)[0]
expression.head()

,TargetID,Gene_Symbol,Chr,Coord,sample_id,expression
0,ENSG00000232629.4,ENSG00000232629.4,6,32731311,HG00096,9.96245
1,ENSG00000179344.11,ENSG00000179344.11,6,32636160,HG00096,43.78876
2,ENSG00000176681.9,ENSG00000176681.9,17,44370099,HG00096,6.34823
3,ENSG00000214425.1,ENSG00000214425.1,17,43595264,HG00096,0.40868
4,ENSG00000237541.3,ENSG00000237541.3,6,32709119,HG00096,51.03589


Prepare data for Expression prediction.

y = expression values for (sample, gene) pairs.

X = matrix containing the embeddings for each (sample, gene) pair.

In [7]:
df = pd.merge(df, expression, left_on=["sample_id", "gene_id"], right_on=["sample_id", "Gene_Symbol"])
print(f"Number of samples for prediction expression: {len(df)}")
genes = df.gene_id.unique()
print(f"Number of genes: {len(genes)}")

Number of samples for prediction expression: 4830
Number of genes: 10


In [78]:
def train_model(df, gene, method="linear", k=5, n_iter_search=25):
    if method not in ["linear", "ridge", "lasso", "elasticnet", "svr"]:
        raise ValueError(f"Method {method} is not valid, must be linear, ridge, lasso, elasticnet or svr!")
    
    # prepare data
    y = df[df.gene_id==gene].expression.values
    X = np.stack(df[df.gene_id==gene].embedding.values)
    
    # setup k-fold
    kf = KFold(n_splits=k, shuffle=True, random_state=83)
    
    models = []
    results = []
    # iterate over k folds
    # print(f"Results (rvalues) of k-fold regression for gene {gene} using a {method} model:")
    for i, (train, test) in enumerate(kf.split(X)):
        X_train, X_test, y_train, y_test = X[train], X[test], y[train], y[test]

        # train the model
        if method=="linear":
            model = LinearRegression().fit(X_train, y_train)
        elif method=="ridge":
            # model = Ridge(alpha=0.05).fit(X_train, y_train)
            # hyperparameter tuning for alpha
            param_dist = {"alpha": stats.loguniform(1e-4, 1e1)}
            model = RandomizedSearchCV(Ridge(), param_distributions=param_dist, n_iter=n_iter_search)
            model.fit(X_train, y_train) 
        elif method=="lasso":
            # model = Lasso(alpha=1.0).fit(X_train, y_train)
            # hyperparameter tuning for alpha
            param_dist = {"alpha": stats.loguniform(1e-4, 1e1)}
            model = RandomizedSearchCV(Lasso(), param_distributions=param_dist, n_iter=n_iter_search)
            model.fit(X_train, y_train) 
        elif method=="elasticnet":
            model = ElasticNet(random_state=53).fit(X_train, y_train)
        else:
            # model = make_pipeline(StandardScaler(), SVR(C=1, epsilon=0.1)).fit(X_train, y_train)
            # hyperparameter tuning for C, epsilon, kernel, gamma
            param_dist = {"C": stats.expon(scale=100),
                          "epsilon": stats.expon(scale=10),
                          "kernel": ["rbf", "poly", "sigmoid"],
                          "gamma": stats.expon(scale=.1)}
            # model = RandomizedSearchCV(make_pipeline(StandardScaler(), SVR()), param_distributions=param_dist, n_iter=n_iter_search)
            model = RandomizedSearchCV(SVR(), param_distributions=param_dist, n_iter=n_iter_search)
            model.fit(X_train, y_train) 
        
        # evaluation
        predicted = model.predict(X_test)
        evaluation = pearsonr(y_test, predicted)[0]
        results.append(evaluation)
        models.append(model)
        # print(f"Fold {i}: rvalue = {evaluation}")
    return models, results

In [33]:
# train a linear model for each gene:
print(f"Gene\t\t\tMean rvalue over k splits")
for gene in genes:
    models, results = train_model(df, gene)
    print(f"{gene}:\t{np.mean(results)}")

Gene			Mean rvalue over k splits
ENSG00000198502.5:	0.2790504866432765
ENSG00000214425.1:	0.12236253915457777
ENSG00000238083.3:	0.2968507874251478
ENSG00000229450.2:	0.2731398749218303
ENSG00000179344.11:	0.1129021632976066
ENSG00000237541.3:	0.06908194208150492
ENSG00000196126.6:	0.1752590153235853
ENSG00000196735.6:	0.009723140355579302
ENSG00000176681.9:	0.1248767907710405
ENSG00000232629.4:	0.19893528632961813


In [79]:
# generate results for each method and each gene
k = 5

m = []
g = []
i = []
r = []
# all_methods = ["linear", "ridge", "lasso", "elasticnet", "svr"]
all_methods = ["svr"]
for method in all_methods:
    for gene in genes:
        models, results = train_model(df, gene, method, k=k)
        m += [method]*k
        g += [gene]*k
        i += [x for x in range(1,k+1)]
        r += results
result_df = pd.DataFrame({"method": m, "gene": g, "fold": i, "rvalue": r})

In [80]:
result_df

,method,gene,fold,rvalue
0,svr,ENSG00000198502.5,1,0.352221
1,svr,ENSG00000198502.5,2,0.319667
2,svr,ENSG00000198502.5,3,0.323881
3,svr,ENSG00000198502.5,4,0.295990
4,svr,ENSG00000198502.5,5,0.466773
5,svr,ENSG00000214425.1,1,0.285415
6,svr,ENSG00000214425.1,2,0.379329
7,svr,ENSG00000214425.1,3,NaN
8,svr,ENSG00000214425.1,4,NaN
9,svr,ENSG00000214425.1,5,0.332837


### SVR experiments

In [94]:
gene = genes[0]
y = df[df.gene_id==gene].expression.values
X = np.stack(df[df.gene_id==gene].embedding.values)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

param_dist = {"C": stats.expon(scale=100),
              "epsilon": stats.expon(scale=10),
              "gamma": stats.expon(scale=.1)}

# run randomized search
n_iter_search = 25
for kernel in ["rbf", "poly", "sigmoid"]:
    random_search = RandomizedSearchCV(SVR(kernel=kernel), param_distributions=param_dist, n_iter=n_iter_search)
    random_search.fit(X_train, y_train)

    predicted = random_search.predict(X_test)
    print(f"R² of test set: {r2_score(y_test, predicted)}")

R² of test set: 0.24118409589855738
R² of test set: 0.1280974506578313
R² of test set: 0.14904481811495363


In [ ]:
# Question: how to best set the distributions?

# gene      rbf     poly       sigmoid              winner
# 1         0.23    0.08       0.12                 rbf
# 2         0.08    0.10       0.18                 sig
# 3         0.24    0.16       0.28                 sig
# 4         -1.42   -0.49      -0.09    ???         -
# 5         0.38    0.49       0.19                 poly
# 6         0.25    0.11       0.07                 rbf
# 7         0.21    0.10       0.04                 rbf
# 8         0.23    0.31       0.23                 poly
# 9         0.20    0.14       0.07                 rbf
# 10        0.33    0.26       0.22                 rbf

# => rbf wins 5 times, poly 2 times and sig 2 times.

# Comparison to prediction based on single SNPs

In [22]:
reference = pd.read_csv("datasets/EUR373.gene.cis.FDR5.all.rs137.txt.gz", sep="\t")
reference

,SNP_ID,ID,GENE_ID,PROBE_ID,CHR_SNP,CHR_GENE,SNPpos,TSSpos,distance,rvalue,pvalue,log10pvalue
0,rs1566523,-,ENSG00000142794.11,ENSG00000142794.11,1,1,21766663.0,21766621,42.0,-0.782786,3.883913e-78,77.410730
1,rs66732917,-,ENSG00000142794.11,ENSG00000142794.11,1,1,21766608.0,21766621,13.0,-0.775971,5.736684e-76,75.241339
2,rs12734589,-,ENSG00000142794.11,ENSG00000142794.11,1,1,21760127.0,21766621,6494.0,-0.753190,3.094123e-69,68.509462
3,rs4654745,-,ENSG00000142794.11,ENSG00000142794.11,1,1,21767096.0,21766621,475.0,-0.753071,3.339200e-69,68.476358
4,rs10916990,-,ENSG00000142794.11,ENSG00000142794.11,1,1,21763132.0,21766621,3489.0,-0.750913,1.327049e-68,67.877113
...,...,...,...,...,...,...,...,...,...,...,...,...
419978,rs133379,-,ENSG00000198951.6,ENSG00000198951.6,22,22,42468535.0,42466846,1689.0,0.231022,6.568113e-06,5.182559
419979,rs4253766,-,ENSG00000075234.12,ENSG00000075234.12,22,22,46623905.0,46663858,39953.0,-0.231008,6.576526e-06,5.182003
419980,rs73165152,-,ENSG00000100417.7,ENSG00000100417.7,22,22,42340397.0,41985894,354503.0,-0.231006,6.577641e-06,5.181930
419981,rs133352,-,ENSG00000198951.6,ENSG00000198951.6,22,22,42434022.0,42466846,32824.0,0.231003,6.579434e-06,5.181811
